# Bike

In [ ]:
import sage
import pickle
import baselines
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.bike()
feature_names = df.columns.tolist()[:-3]

In [ ]:
# Split data, with total count serving as regression target
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=123)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=123)
Y_train = train[:, -1].copy()
Y_val = val[:, -1].copy()
Y_test = test[:, -1].copy()
train = train[:, :-3].copy()
val = val[:, :-3].copy()
test = test[:, :-3].copy()

In [ ]:
with open('trained_models/bike model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
for i in range(512):
    bike_scores = baselines.permutation_test(
        model, test, Y_test, 'mse', n_permutations=1)
    bike_dict = {
        'scores': bike_scores,
        'evals': len(test)
    }
    with open('results/bike permutation_test {}.pkl'.format(i), 'wb') as f:
        pickle.dump(bike_dict, f)
    print('Done with {}'.format(i))

# Bank

In [ ]:
import sage
import pickle
import baselines
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.bank()

# Feature names and categorical columns (for CatBoost model)
feature_names = df.columns.tolist()[:-1]
categorical_cols = ['Job', 'Marital', 'Education', 'Default', 'Housing',
                    'Loan', 'Contact', 'Month', 'Prev Outcome']
categorical_inds = [feature_names.index(col) for col in categorical_cols]

In [ ]:
# Split data
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=123)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=123)
Y_train = train[:, -1].copy().astype(int)
Y_val = val[:, -1].copy().astype(int)
Y_test = test[:, -1].copy().astype(int)
train = train[:, :-1].copy()
val = val[:, :-1].copy()
test = test[:, :-1].copy()

In [ ]:
with open('trained_models/bank model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
for i in range(512):
    bank_scores = baselines.permutation_test(
        model, test, Y_test, 'cross entropy', n_permutations=1)
    bank_dict = {
        'scores': bank_scores,
        'evals': len(test)
    }
    with open('results/bank permutation_test {}.pkl'.format(i), 'wb') as f:
        pickle.dump(bank_dict, f)
    print('Done with {}'.format(i))

# Credit

In [ ]:
import sage
import pickle
import baselines
import numpy as np
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
df = sage.datasets.credit()

# Feature names and categorical columns (for CatBoost model)
feature_names = df.columns.tolist()[:-1]
categorical_columns = [
    'Checking Status', 'Credit History', 'Purpose', 'Credit Amount',
    'Savings Account/Bonds', 'Employment Since', 'Personal Status',
    'Debtors/Guarantors', 'Property Type', 'Other Installment Plans',
    'Housing Ownership', 'Job', 'Telephone', 'Foreign Worker'
]
categorical_inds = [feature_names.index(col) for col in categorical_columns]

In [ ]:
# Split data
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=0)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=0)
Y_train = train[:, -1].copy().astype(int)
Y_val = val[:, -1].copy().astype(int)
Y_test = test[:, -1].copy().astype(int)
train = train[:, :-1].copy()
val = val[:, :-1].copy()
test = test[:, :-1].copy()

In [ ]:
with open('trained_models/credit model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
for i in range(512):
    credit_scores = baselines.permutation_test(
        model, test, Y_test, 'cross entropy', n_permutations=1)
    credit_dict = {
        'scores': credit_scores,
        'evals': len(test)
    }
    with open('results/credit permutation_test {}.pkl'.format(i), 'wb') as f:
        pickle.dump(credit_dict, f)
    print('Done with {}'.format(i))

# BRCA

In [ ]:
import sage
import pickle
import baselines
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
gene_names = [
    'BCL11A', 'IGF1R', 'CCND1', 'CDK6', 'BRCA1', 'BRCA2', 'EZH2', 'SFTPD',
    'CDC5L', 'ADMR', 'TSPAN2', 'EIF5B', 'ADRA2C', 'MRCL3', 'CCDC69', 'ADCY4',
    'TEX14', 'RRM2B', 'SLC22A5', 'HRH1', 'SLC25A1', 'CEBPE', 'IWS1', 'FLJ10213',
    'PSMD10', 'MARCH6', 'PDLIM4', 'SNTB1', 'CHCHD1', 'SCMH1', 'FLJ20489',
    'MDP-1', 'FLJ30092', 'YTHDC2', 'LFNG', 'HOXD10', 'RPS6KA5', 'WDR40B',
    'CST9L', 'ISLR', 'TMBIM1', 'TRABD', 'ARHGAP29', 'C15orf29', 'SCAMP4',
    'TTC31', 'ZNF570', 'RAB42', 'SERPINI2', 'C9orf21'
]

In [ ]:
# Load data.
expression = pd.read_table('data/BRCA_TCGA_microarray.txt',
                           sep='\t', header=0,
                           skiprows=lambda x: x == 1, index_col=0).T
expression.index = pd.Index(
    ['.'.join(sample.split('-')[:3]) for sample in expression.index])

# Filter for reduced gene setif reduced:
expression = expression[gene_names]

# Impute missing values.
expression = expression.fillna(expression.mean())

# Load labels.
labels = pd.read_table('data/TCGA_breast_type.tsv',
                       sep='\t', header=None,
                       index_col=0, names=['Sample', 'Label'])

# Filter for common samples.
expression_index = expression.index.values
labels_index = labels.index.values
intersection = np.intersect1d(expression_index, labels_index)
expression = expression.iloc[[i for i in range(len(expression))
                              if expression_index[i] in intersection]]
labels = labels.iloc[[i for i in range(len(labels))
                      if labels_index[i] in intersection]]

# Join expression data with labels.
label_data = labels['Label'].values
label_index = list(labels.index)
expression['Label'] = np.array(
    [label_data[label_index.index(sample)] for sample in expression.index])
expression['Label'] = pd.Categorical(expression['Label']).codes
data = expression.values

# Split data
train, test = train_test_split(
    data, test_size=int(0.2 * len(data)), random_state=0)
train, val = train_test_split(
    train, test_size=int(0.2 * len(data)), random_state=0)
Y_train = train[:, -1].copy().astype(int)
Y_val = val[:, -1].copy().astype(int)
Y_test = test[:, -1].copy().astype(int)
train = train[:, :-1].copy()
val = val[:, :-1].copy()
test = test[:, :-1].copy()

# Preprocess
mean = train.mean(axis=0)
std = train.std(axis=0)
train = (train - mean) / std
val = (val - mean) / std
test = (test - mean) / std

In [ ]:
with open('trained_models/brca model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
for i in range(512):
    brca_scores = baselines.permutation_test(
        model, train, Y_train, 'cross entropy', n_permutations=1)
    brca_dict = {
        'scores': brca_scores,
        'evals': len(train)
    }
    with open('results/brca permutation_test {}.pkl'.format(i), 'wb') as f:
        pickle.dump(brca_dict, f)
    print('Done with {}'.format(i))

# MNIST

In [ ]:
import sage
import torch
import matplotlib.pyplot as plt
import torchvision.datasets as dsets

In [ ]:
# Load train set
train = dsets.MNIST('../data', train=True, download=True)
imgs = train.data.reshape(-1, 784) / 255.0
labels = train.targets

# Shuffle and split into train and val
inds = torch.randperm(len(train))
imgs = imgs[inds]
labels = labels[inds]
val, Y_val = imgs[:6000], labels[:6000]
train, Y_train = imgs[6000:], labels[6000:]

# Load test set
test = dsets.MNIST('../data', train=False, download=True)
test, Y_test = test.data.reshape(-1, 784) / 255.0, test.targets

# Move test data to numpy
test_np = test.cpu().data.numpy()
Y_test_np = Y_test.cpu().data.numpy()

In [ ]:
device = torch.device('cuda', 2)
model = torch.load('trained_models/mnist mlp.pt')
model = model.to(device)

In [ ]:
for i in range(1024):
    mnist_scores = baselines.permutation_test(
        model, test_np, Y_test_np, 'cross entropy', n_permutations=1)
    mnist_dict = {
        'scores': mnist_scores,
        'evals': len(test_np)
    }
    with open('results/mnist permutation_test {}.pkl'.format(i), 'wb') as f:
        pickle.dump(mnist_dict, f)
    print('Done with {}'.format(i))